# Regression Analysis - A Plan for This Lesson

We're going to learn a little bit about traditional statistical analyses.  The crucial piece here is not that you to fully understand all of traditional statistics -- that's another class -- but rather to gain a sense of what it can (and more importantly, what it *can not*) do.  

Traditional statistical analyses are how many people do exploratory research.  But do not make the classic research mistake of thinking that patterns that you find in your data represent reality. Regression analyses are powerful, but inept use of regressions also lays the groundwork for much of the replication crisis currently engulfing the social sciences.  If you haven't heard about [data dredging](https://en.wikipedia.org/wiki/Data_dredging), p-hacking and over-fitting, get used to those terms.  They are bane of modern research existence, and they are still really, really widespread.  More than that, even when using the best machine learning practices, people make all kinds of mistakes about what data should and should not be included in their particular application.  We'll talk a bit more about that as well. 

That plan for this lesson is to get some data sets loaded, make sure they're all properly geocoded and merged, and then look for patterns in the data.  We'll find one -- but it might not be as simple as it appears at first. 


# Startup - Installing and Importing

First things first: let's get all our packages installed and imported.  You'll see that we need to install geopandas and xtree. thankfully, statsmodels (the package we'll use to run our regressions) and seaboarn (the package we'll use to make some pretty plots) are both already installed, and we can just import them. 

In [ ]:
! conda install geopandas -yq
import pandas as pd
import geopandas as gpd
import numpy as np
import statsmodels.api as sm 
import statsmodels.formula.api as smf 
import matplotlib.pyplot as plt
% matplotlib  inline
import seaborn as sns; sns.set(color_codes=True)

# 01 - reading in data

Let's read in some District crime data. Eventually we'll deal with crime & time in a more complex manner. 
For now, though, let's just have a look crime from a single year: 2010. Find the data on Open Data DC. Instead of downloading the data to your laptop, just copy the url for the "spreadsheet" under the download menu. Pandas is smart enough to just import it via the static url from that site. In a single line, read the data into a dataframe called "crimes" by using the pd.read_csv() function, and the url you just copied (inside quotation marks, because it's a string). 

In [ ]:
crimes = pd.read_csv('https://opendata.arcgis.com/datasets/fdacfbdda7654e06a161352247d3a2f0_34.csv')

That was easy! One line and we have some great data from the web!  You'll see that we can't always do that, but it's nice when we can. 

# 02 - exploring the data

Ok, let's see what's in this data. 

In [ ]:
print(crimes.columns)

It looks like the types of offenses recorded here are probably labeled ... OFFENSE!  As we've seen before, 
one nice way to get a sense of the frequencies in our data is to use .value_counts().  Let's see how many crimes were committed in each ward. 

In [ ]:
print(crimes.WARD.value_counts())

That's a mix of every kind of crime. Let's see if we can focus on violent crimes.  To figure out which of those we have in this data set, print out the value counts for the column named "OFFENSE". 

It looks like the most violent offenses are, thankfully, also among the least common. But assoault with a dangerous weapon is still pretty common.  Let's focus on violent crimes against persons.  

# Subsetting
Although the final statement might seem complex, it's actually fairly straightforward once you get the hang of it.  


## 03 - True and false

First, let's create a set of logical statments that give us a "True" for each of the crimes we want.  

```crimes['OFFENSE']=='HOMICIDE'```

That is a piece of code that asks pandas "does the values in the column OFFENSE equal 'HOMICIDE', and it will return True or False for every single row in the dataframe. Try it out. 

Why is that helpful?  Because if we wanted to create a new dataframe with just homicides, we can now do the following: 

``` crimes[crimes['OFFENSE']=='HOMICIDE'] ```

Try it! 

Do you see how we nested that line from above inside the square brackets after the crimes dataframe?  That tells pandas to return only those rows where the statement is true. And since it is only true where the OFFENSE is HOMICIDE, it is taking all the homicide rows from the crimes dataframe and copying them into the new homicide dataframe. 

But... we really want more than homicides.  So, we're going to combine three statments together: 

```
crimes['OFFENSE']=='HOMICIDE'
crimes['OFFENSE']=='ASSAULT W/DANGEROUS WEAPON'
crimes['OFFENSE']=='SEX ABUSE'

```
And we'll link them with a funny symbol that means OR in python: |.  It's that straight line just above the downward slash over the enter/return key. 

In [ ]:
crimes = crimes[(crimes['OFFENSE']=='HOMICIDE') | 
                (crimes['OFFENSE']=='ASSAULT W/DANGEROUS WEAPON') |
                (crimes['OFFENSE']=='SEX ABUSE')]
print(crimes.OFFENSE.unique())

Now we have our violent crimes against persons in DC from 2010. Time to add some geography. 

# 04 - adding census data and geography

To read in our census data, I like to use geojson.  Geojson combines the shape data with other data in a single file. It tends to be large, but it's extremely convenient.  Let's try grabbing it from Opend data DC .  

In [ ]:
tracts = gpd.read_file('https://opendata.arcgis.com/datasets/6969dd63c5cb4d6aa32f15effb8311f3_8.geojson')

What?  I though pandas could read in data stright from from a URL?  Well: pandas can; geopandas can't.  We'll have to download this before loading into geopandas. But let's do this like the pros do.  To grab the data (the URL is available under the "APIs" dropdown menu as "geojson"), we'll use the fantastic unix command "wget".  We can take the crazy name in the URL and given a more reasonable name by using the -O option, thusly: 

In [ ]:
! wget https://opendata.arcgis.com/datasets/6969dd63c5cb4d6aa32f15effb8311f3_8.geojson -O census.geojson

# 05 - reading the data into geopandas

OK, recall that reading in to geopandas is as easy as ```gpd.read_file()```. Load that file named "census.geojson" into a geopandas dataframe called "census".  It's already in your working direcory in the cloud thanks to wget.  :-)

Nice.  Now let's see what the data looks like. 

In [ ]:
census.columns

Blargh.  Why did the census name everything with code?  I guess because computers used to have tiny, tiny eight character naming limits. Well, I'll tell you what.  FAGI is the "Federal Adjusted Gross Income", and it's a very valuable little piece of data to have. There are also data on race, ethnicity and -- if we do a little math -- population density in here.  

But before we get into all that, let's take the data for violent crimes in 2010 and plot it inside our census tracts like we learned last week. 

In [ ]:
from shapely.geometry import Point
geometry = [Point(xy) for xy in zip(crimes.LONGITUDE.apply(float), crimes.LATITUDE.apply(float))]
crs = {'init': 'epsg:4326'}
points = gpd.GeoDataFrame(crimes, crs=crs, geometry=geometry)
fig, ax = plt.subplots()
census.plot(ax=ax, color='red')
points.plot(ax=ax, color='black', marker='.', markersize=5)
ax.set_aspect('equal')  # remember, this is what we need to do in order to not have a crazy wide map. 

We still need to aggregate the crime into each census tract, and to do that we need to join that crime data with the census tract data. 

In [ ]:
geo_crimes = gpd.sjoin(census, points, how='left', op='intersects')
geo_crimes.head()

Now we have the data in pretty much the format we want.  We have a bunch of census tracts wtih crimes aggregated.  But what we still don't know is how many crimes are committed *per capita*, which is far more informative than just how many crimes have occured. For that we need to know the total population in each census tract.  And that is actually in the column named "P0010001". What a great name.  The best.  All right, putting it all together, just like we did last week, we get: 

In [ ]:
crimes_percap = pd.DataFrame(geo_crimes.OBJECTID_left.value_counts()*100000/geo_crimes.P0010001.sum())
crimes_percap.columns = ['crimes_percap']
tracts_crimes = census.merge(crimes_percap, how="left", left_on='OBJECTID', right_index=True)
tracts_crimes.plot(column='crimes_percap', cmap='YlOrRd')

# 06 - drawing some dots and some lines

As this lesson is about the traditional statistical technique of regression, we're going to do what researchers did for ages before they had machine learning algorithms: look at some simple patterns.  If X and Y are positively correlated, that means that the higher X is, at least on average, the higher Y is.  And if X and Y are negatively correlated, when X goes higher, Y goes lower. 

Let's draw a fairly simple plot, then we can try to see if there is a pattern we can discern.  Try this: 



In [ ]:
df = pd.DataFrame(tracts_crimes[['FAGI_MEDIAN_2010', 'crimes_percap']])
df.plot.scatter('FAGI_MEDIAN_2010', 'crimes_percap')

That seems like a pretty steep falloff as income goes up.   

If you drew a line through those dots, what would it look like?  We'll try a few statistical technicques, but you could probably just draw a line with you finger on the screen. Do it slant upward or downward as you move from left to right?  

Better than imagining a line, we could actually draw using math.  Or, better still, let the comupter draw one using math!  To do this, we'll use the delightful package called "seaborn" that we loaded at the top of the page. 

Seaborn is going to fit a line using simple linear regression, a line that has the least "residual error" possible.  This means that if you moved the line in any way, the overall sum of the distances between the dots and the line with INCREASE.  

In [ ]:
ax = sns.regplot(x="FAGI_MEDIAN_2010", y="crimes_percap", data=tracts_crimes)

That line is as close to ALL the dots as a straight line can be.  This is known as linear regression. And we can actually figure out a lot fo things about how good a fit it is if we use the statsmodels package.  

Let's fit a model that has just those two variables: income and crime rate. 

In [ ]:
model = 'crimes_percap ~ FAGI_MEDIAN_2010'

results = smf.ols(model, data=tracts_crimes).fit()

print(results.summary())

So from this output we can learn a lot.  First, the R-squared tells us about what percentage of the variation in the crime rate is also associated with variation in median income.  So there's roughly at 32% association.  That's better than nothing, but pretty far short of 100%.  And, if you are like puzzles, you'll be tempted to see if you can "explain" more of the variation.  That's good -- but also potentially bad. 

Let's see what we can do with a few reasonable adjustments. First, if you were drawing a line through those dots, and you could bend it a bit, would it be a better fit?  Probably, right?  the dots all seem to cluster near the axes.  In fact, if we want to, we can check this intuition by drawing what is called a "local fit" regression line. This is often referred to as "loess" or "lowess".  And seaborn has a very nice one.  

In [ ]:
ax = sns.regplot(x="FAGI_MEDIAN_2010", y="crimes_percap", data=tracts_crimes)
sns.regplot(x="FAGI_MEDIAN_2010", y="crimes_percap", data=tracts_crimes, lowess=True)

When a statistician sees a plot like this, they think: LOG scale.  A log scale just means reducing a number by a factor. So: 

```
number  log
0.001   -3
0.01    -2
0.1     -1
1        0
10       1
100      2
1000     3
```

The highly technical term for this that I like to use is "unsmooshing".  Let's try that here by applying a log transformation to each of the variables. 

In [ ]:
tracts_crimes['income_log'] = tracts_crimes['FAGI_MEDIAN_2010'].apply(np.log)
tracts_crimes['crime_log'] = tracts_crimes['crimes_percap'].apply(np.log)

Now take the same from above (the two lines for the seaborn plots) and subsitute the variables we just recreated.  

Notice that there is actually very little difference between straight line in the linear regression (blue), and the "local fit" line in the the lowess regression (green). So it looks like we were correct -- this is a log distribution in both directions.  You'll find that income data are almost always like this: people tend to cluster at the bottom of the scale, and at the top people are really, really rich. This pattern is true of many other measures as well. 

In human terms, what this means is that the dropoff in the crime rate associated with increased median income in a census tract are both very steep, but grows less steep over time. Increasingly larger amounts of income (log of the income) are correlated with a increasingly smaller reductions in the violent crime rate (log of crime percapita).  Another way of putting it is that the loss of each subsequent dollar of income is associated with a substantially larger increase in violent crime.  Many data sets contain log distributions -- that is, they are both concentrated at the low end of the scale. 

And, indeed, this does make for a better statistical fit when we have a look at the regression statistics in statsmodels: 

In [ ]:
model = 'np.log(crimes_percap) ~ np.log(FAGI_MEDIAN_2010) '

results = smf.ols(model, data=tracts_crimes).fit()

print(results.summary())

Indeed, median income appears to explain away nearly 50% of the variation in violent crime.  Or is it the other way around?  Does poverty lead to violence, or does violence drive away money?  It's not an idle question, and a great deal of thought needs to go into the answer before we even begin to think about using this model to inform a particular public policy or practice. 

There are other ways of increasing fit, and we could have chosen from many different types of transformations or models of the data.  

# Danger, Will Robinson, Danger!

But for the reckless, why stop at one variable?  The world is full of data, and researchers have thrown a lot of data crime to see what is correlated and what drives up their R-squared. I'll create a few more variables to add to the mix, and you can play around with the formulas in the equation.  

In [ ]:
tracts_crimes['percent_white'] = tracts_crimes['P0010003'] / tracts_crimes['P0010001']
tracts_crimes['percent_black'] = tracts_crimes['P0010004'] / tracts_crimes['P0010001']
tracts_crimes['percent_asian'] = tracts_crimes['P0010006'] / tracts_crimes['P0010001']
tracts_crimes['population_density'] = tracts_crimes['P0010001'] / tracts_crimes['SQ_MILES']

In [ ]:
model = 'crime_log ~ income_log + population_density + percent_white + percent_black + percent_asian '
results = smf.ols(model, data=tracts_crimes).fit()
print(results.summary())

# crazy interactions

Just to show you how crazy these models can get, I'll fully interact all the variables, meaning that I'm assuming all the variables interact with each other in a way that matters to crime.  I have not real theory -- indeed I doubt one could come up with a reasonable one -- but you should know that p-hackers tend to interact almost everthing, then drop the less promsing interactions.  The theory the data supports is then invented after they have found novel pattern in the data. Publication gold! But terrible for science and even worse for public policy. 

To commit this statistical crime, run the same commands, but inside the formual, replace the plus sign with an asterisk (\* instead of +)

# errr...

But, hey, our R-squared keeps going up ... we must be doing something right!  



# automating the data dredge

Sometimes -- not when trying to make any claims about causation -- but sometimes when doing exploratory analyses, you might just want to automate your data dredging workflow.  Many statistical packages actually have something called "stepwise regression" that does this for the researcher.  Stepwise regression is meant to be exploratory, but because almost no one who publishes results pre-registers their experiments it has become a mainstay of the academic publishing dystopia.  People get data, run a bazillion transformations and interactions, then run stepwise regression to maximize their R-squared, increase their confidence intervals, and come out with a totally confabulated hypothesis that their analysis "tests".  

In future weeks, we'll start to see how this kind of automation, when done correctly, can be a good thing.  But for now, let's just see how we would do this in Python, which does not have a package to do this.  We'll just make a simple function that can iterate through a fully interacted model.  

You don't have to understand all the code to know this: many researchers do things like this all the time, building up extremely fragile models that increase the fit fo the their model, but have no theory and are never really thought through.  Needless to say, you shouldn't do it.  But with that advice, let's do something really stupid, so we can learn a bit about how sci-kit learn work later on. 

In [ ]:

def data_mining(data, dv):
    remaining = set(data.columns)
    remaining.remove(dv)
    selected = []
    current_score, best_new_score = 0.0, 0.0
    while remaining and current_score == best_new_score:
        scores_with_candidates = []
        for candidate in remaining:
            formula = "{} ~ {} + 1".format(dv, ' * '.join(selected + [candidate]))
            score = smf.ols(formula, data).fit().rsquared_adj
            scores_with_candidates.append((score, candidate))
        scores_with_candidates.sort()
        best_new_score, best_candidate = scores_with_candidates.pop()
        if current_score < best_new_score:
            remaining.remove(best_candidate)
            selected.append(best_candidate)
            current_score = best_new_score
    formula = "{} ~ {} + 1".format(dv, ' * '.join(selected))
    model = smf.ols(formula, data).fit()
    return model



In [ ]:
myvars = ['crime_log', 'income_log', 'population_density', 'FAGI_TOTAL_2010']

df = pd.DataFrame(tracts_crimes[myvars])
mining = forward_selected(df,'crime_log')
mining.summary()

# adding more variables
Ok, now add the following to our list of variables to myvars: 
```
'percent_asian', 'percent_white', 'percent_black'
```

Hey, the R-squared went up again!  We must be geniuses.  

OK, enough of the bad stats practices. We'll soon see how to turn these to the power of good.  But please just remember: with great power...